In [1]:
# ! export MACOSX_DEPLOYMENT_TARGET=10.9
# ! pip install extremetext

In [1]:
# ! pip install scikit-multilearn

In [2]:
# https://github.com/mwydmuch/extremeText
# https://arxiv.org/pdf/1810.11671v1.pdf

In [133]:
import pandas as pd
import extremeText
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.datasets import make_multilabel_classification

In [145]:
df = pd.read_csv('../data/2019-05-28_portuguese_hate_speech_hierarchical_classification.csv')
print(df.shape)
df.head(2)

(5668, 80)


,text,Hate.speech,Sexism,Body,Racism,Ideology,Homophobia,Origin,Religion,Health,...,Thin.women,Arabic,East.europeans,Africans,South.Americans,Brazilians,Migrants,Homossexuals,Thin.people,Ageing
0,"""não come mel, morde marimbondo""",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"não tem pinto, tem orgulho !",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Preprocessing

In [150]:
import nltk
from nltk.corpus import stopwords
pt_stopwords = stopwords.words('portuguese')
def stop_words(value):
    for text in value:      
        tokens = nltk.word_tokenize(text.lower())
    return tokens

# df['text'] = df.text.apply(stop_words)

In [152]:
def preprocessing():
    # remove mentions
    # lower case
    return None

In [147]:
df.text

0                        "não come mel, morde marimbondo"
1                            não tem pinto, tem orgulho !
2       Não vê essa merda de Crepúsculo! Pra isso temo...
3       não da tapa na bundinha, da cotovelada nas cos...
4       o diminutivo INHO não acompanha a trajetória d...
                              ...                        
5663    Na minha sala só tem viado e sapatão  e a cois...
5664    PARABENS SAPATÃO SDDS @attomiter https://t.co/...
5665    RT @toquedeveludo: Agora um poema:\nEu sou sap...
5666    O mundo das sapatao é mais ligado do que eu im...
5667    Gente, sapatão é uma coisa q me assusta https:...
Name: text, Length: 5668, dtype: object

In [149]:
df.text.apply(stop_words)

0       [``]
1        [!]
2        [.]
3        [!]
4        [.]
        ... 
5663     [o]
5664     [c]
5665      []
5666     [a]
5667     [g]
Name: text, Length: 5668, dtype: object

In [131]:
df.head()

,text,Hate-speech,Sexism,Body,Racism,Ideology,Homophobia,Origin,Religion,Health,...,Africans,South-Americans,Brazilians,Migrants,Homossexuals,Thin-people,Ageing,label_total,label_binary,extremeText_label
0,[``],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['``']
1,[!],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['!']
2,[.],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['.']
3,[!],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['!']
4,[.],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['.']


In [130]:
# regex
# essa, esta, essas, estas, esse, esses, este, estes: es[st]?*
stopwords_manual = ['http?', 'mais', 'is?o', 'es[st]?*', 'quan[dt]?']

In [104]:
# Data should follow this format from extreme text

train_data_format = """
__label__mariadb-galera __label__mariadb55-mariadb __label__mysql55-mysql mariadb mariadb mysql solaris vulnerability oracle mysql server users availability vectors keys oracle com technetwork topics security html http secunia com http www oracle com technetwork topics security http lists security announce msg00016 html http www oracle com technetwork topics security html http secunia com http www securityfocus security gentoo glsa xml mariadb-galera mariadb55-mariadb-devel ruby-mysql openshift-origin-cartridge-mysql rh-mariadb100-mariadb mariadb-apb-role query-mysql mariadb55-mariadb-test rh-mysql57-mysql rh-mariadb101-mariadb rh-mysql56-mysql mysql mysql-connector-java mariadb55-mariadb-bench mysql55-mysql mysql-apb-role mysql mariadb55-mariadb-server mysql-binuuid-rails rh-mysql80-mysql com.github.brandtg switchboard-mysql rh-mariadb102-mariadb mariadb mariadb55-mariadb rhn-solaris-bootstrap mariadb55-mariadb-libs
"""
# https://github.com/automated-library/ICPC_2022_Automated-Identification-of-Libraries-from-Vulnerability-Data/tree/main/extremeText/dataset

In [105]:
# replace "." with "-"
df.columns = df.columns.str.replace('.', '-')

cols = df.columns
cols = ['Hate-speech', 'Sexism', 'Body', 'Racism', 'Ideology', 'Homophobia', 'Origin', 'Religion', 'Health', 'OtherLifestyle', 'Aborting-women', 'Agnostic', 'Argentines', 'Asians', 'Autists', 'Black-Women', 'Blond-women', 'Brazilians-women', 'Chinese', 'Criminals', 'Egyptians', 'Fat-people', 'Football-players-women', 'Gamers', 'Homeless', 'Homeless-women', 'Indigenous', 'Iranians', 'Japaneses', 'Jews', 'Jornalists', 'Latins', 'Left-wing-ideology', 'Men-Feminists', 'Mexicans', 'Muslims-women', 'Nordestines', 'Old-people', 'Polyamorous', 'Poor-people', 'Rural-people', 'Russians', 'Sertanejos', 'Street-artist', 'Ucranians', 'Vegetarians', 'White-people', 'Young-people', 'Old-women', 'Ugly-people', 'Venezuelans', 'Angolans', 'Black-people', 'Disabled-people', 'Fat-women', 'Feminists', 'Gays', 'Immigrants', 'Islamists', 'Lesbians', 'Men', 'Muslims', 'Refugees', 'Trans-women', 'Travestis', 'Women', 'Bissexuals', 'Transexuals', 'Ugly-women', 'Thin-women', 'Arabic', 'East-europeans', 'Africans', 'South-Americans', 'Brazilians', 'Migrants', 'Homossexuals', 'Thin-people', 'Ageing']

def label_value(value, col):
    label = f'__label__{col}'
    if value == 1:
        return label
    else:
        return "None"
    
for i in cols:
    df[i] = df[i].apply(label_value, args=(i,))
    
df.Sexism.value_counts()

/var/folders/3k/n8m_c1vj5gb6gyfmb_vsyf9w0000gn/T/ipykernel_5136/1666610618.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('.', '-')


None    5668
Name: Sexism, dtype: int64

In [106]:
df['label_total'] = df[cols].agg(''.join, axis=1)
df['label_total'] = df['label_total'].str.replace("None", "")
df['label_total'] = df['label_total'].str.replace("__label", " __label")
df['label_total'] = df['label_total'].str.strip()

In [107]:
# looks like not all data has a label
# this function identify the one who got a label, and the one who don't
def identify_label(value):
    if "__label" in value:
        return 1
    else:
        return 0

In [108]:
df['label_binary'] = df['label_total'].apply(identify_label)
df['label_binary'].value_counts()

0    5668
Name: label_binary, dtype: int64

In [109]:
df['extremeText_label'] = df['label_total'].astype(str) + df['text'].astype(str)

In [110]:
df['extremeText_label']

0       ['``']
1        ['!']
2        ['.']
3        ['!']
4        ['.']
         ...  
5663     ['o']
5664     ['c']
5665        []
5666     ['a']
5667     ['g']
Name: extremeText_label, Length: 5668, dtype: object

In [111]:
labeled_data = list(df[df['label_binary'] == 1]['extremeText_label'].values)

In [112]:
with open('./data/train_hs.txt', 'a') as the_file:
    for item in labeled_data:
        the_file.write(f"{item}\n")

# Split dataset

In [113]:
# SIMPLE train, test split
# create a new one with this: http://scikit.ml/stratification.html
# X = df.copy()
# del X['Hate.speech']
# del X['text']

# y = df['Hate.speech']

# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [114]:
# train test split data
# https://datascience.stackexchange.com/questions/45174/how-to-use-sklearn-train-test-split-to-stratify-data-for-multi-label-classificat

In [115]:
X,Y = make_multilabel_classification(n_samples=300, n_classes=100, n_labels=10)

In [116]:
X.shape

(300, 20)

In [117]:
Y.shape

(300, 100)

In [118]:
X_train, y_train, X_test, y_test = iterative_train_test_split(X,Y,test_size=0.20)

In [119]:
X_train

array([[0., 6., 1., ..., 6., 2., 3.],
       [0., 1., 2., ..., 2., 1., 2.],
       [6., 1., 0., ..., 4., 2., 1.],
       ...,
       [3., 4., 0., ..., 2., 0., 3.],
       [1., 3., 4., ..., 4., 4., 4.],
       [2., 2., 6., ..., 4., 2., 1.]])

In [120]:
df

,text,Hate-speech,Sexism,Body,Racism,Ideology,Homophobia,Origin,Religion,Health,...,Africans,South-Americans,Brazilians,Migrants,Homossexuals,Thin-people,Ageing,label_total,label_binary,extremeText_label
0,[``],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['``']
1,[!],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['!']
2,[.],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['.']
3,[!],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['!']
4,[.],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['.']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5663,[o],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['o']
5664,[c],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['c']
5665,[],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,[]
5666,[a],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,,0,['a']


# Modeling
https://github.com/automated-library/ICPC_2022_Automated-Identification-of-Libraries-from-Vulnerability-Data/blob/main/extremeText/extremetext_train.py

In [121]:
# https://github.com/automated-library/ICPC_2022_Automated-Identification-of-Libraries-from-Vulnerability-Data/blob/main/extremeText/extremetext_train.py

def model_training(train_data):
    # train_supervised uses the same arguments and defaults as the fastText/extremeText cli

    print("Supervised Training")
    # default supervised training
    # model = extremeText.train_supervised(
    #     input=train_data, epoch=100, lr=1.0, wordNgrams=2, verbose=3, minCount=1,
    # )

    # paper supervised training
    model = extremeText.train_supervised(
        input=train_data, epoch=100, lr=0.05, verbose=3, wordNgrams=2, minCount=1, l2=0.003, arity=2, dim=100, tfidfWeights=True
    )
    model.save_model("./model/xt_supervised.bin")
    return model

    # print("Quantization")
    #
    # model.quantize(input=train_data, qnorm=True, retrain=True, cutoff=100000)
    #
    # model.save_model("model/xt_quantized.ftz")

In [122]:
# model_training(train_data = "./dataset/train.txt")

In [123]:
model = model_training(train_data = "./data/train_hs.txt")

Supervised Training


Training ...
  Model: sup, loss: softmax
  Features: TF-IDF weights, buckets: 2000000

  Update: SGD, lr: 0.050000, L2: 0.003000, dims: 100, epochs: 100, neg: 5
Reading input file ...
Read 0M words
Number of documents: 8036
Number of words: 5887
Number of labels: 911
  Input: 2005887 x 100 (765M)
Setting up loss layer ...
  Output: 911 x 100 (0M)
Starting 12 threads ...
Progress: 100.0% words/sec/thread:   68492 lr:  0.000000 loss:  5.210812 ETA:   0h 0m
Saving model ...


In [124]:
model.predict('O mundo das sapatao é mais ligado')

(('__label__Hate-speech',), array([-0.83271313]))

In [125]:
len(model.get_words())

5887

In [126]:
l_4words = []
for word in model.get_words():
    if len(word) > 3:
        l_4words.append(word)

In [127]:
len(l_4words)

5220

In [129]:
l_4words

['</s>',
 'https',
 'mulher',
 'burra',
 'gorda',
 'mais',
 'feia',
 'sapatão',
 'para',
 'fufas',
 'homem',
 'pode',
 'sapatao',
 'como',
 'mulheres',
 'isso',
 'quando',
 'refugiados',
 'essa',
 'feia,',
 'muito',
 'gente',
 'mesmo',
 'coisa',
 'minha',
 'cara',
 'gorda,',
 'mundo',
 'contra',
 'quem',
 'aqui',
 'gosta',
 'quer',
 'nada',
 '#MulherDeVerdade',
 'fica',
 'você',
 'tudo',
 'assim',
 'acha',
 'feminista',
 'fazer',
 'eles',
 'ainda',
 '@homemdeverdade',
 'menos',
 'sabe',
 'branco',
 'nunca',
 'Mulher',
 'racismo',
 'esse',
 'toda',
 'porque',
 'homens',
 'quero',
 'todo',
 'MULHER',
 'falando',
 'sempre',
 'feliz',
 'vejo',
 'refugiados.',
 'vida',
 'está',
 'fala',
 'pelo',
 'igual',
 'pessoas',
 'chama',
 'tenho',
 'chamar',
 'puta',
 'parece',
 'falar',
 'sobre',
 'seria',
 'pela',
 'negro',
 'hora',
 'casa',
 'feminismo',
 'dizer',
 'feministas',
 'não,',
 'onde',
 'depois',
 'podem',
 'burra,',
 'mulher,',
 'machista',
 'anos',
 'estou',
 'ficar',
 'viado',
 'Como'